In [2]:
import torch
import numpy as np

In [3]:
if torch.cuda.is_available():
    print(True)

True


In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from ray import tune
from ray.tune import CLIReporter
from functools import partial

from ray.tune.schedulers import ASHAScheduler

In [2]:
class Net(nn.Module):
    
    def __init__(self,l1=120, l2=84):
        super(Net, self).__init__()
        print(l1,l2)
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, l1),
            nn.ReLU(),
            nn.Linear(l1, l2),
            nn.ReLU(),
            nn.Linear(l2,10),
                                                                                        
        )
    
    def forward(self,x):
        x= self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [17]:
config = {
    "l1": tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
    "l2": tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
    "lr": tune.loguniform(1e-4, 1e-1),
    "batch_size": tune.choice([2, 4, 8, 16])
}

In [18]:
def custom_dataloader():
    
    training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=transforms.ToTensor()
            )
    
    test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=transforms.ToTensor()
    )
    return training_data, test_data

In [19]:
import torch.optim as optim
from torch.utils.data import random_split


def custom_training(config, checkpoint_dir = None, data_dir = None):
    
    net  = Net(config["l1"], config["l2"])
    
    if torch.cuda.is_available():
        device = "cuda:0"
        
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    else:
        device = "cpu"
        
    net.to(device)
    
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=config["lr"], momentum=0.9)
    if checkpoint_dir:
        model_state, optimizer_state = torch.load(
            os.path.join(checkpoint_dir, "checkpoint"))
        net.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)
    
    
    trainset, testset = custom_dataloader()
    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs])
    
    trainloader = torch.utils.data.DataLoader(
        train_subset,
        batch_size=int(config["batch_size"]),
        shuffle=True,
        num_workers=1)
    
    valloader = torch.utils.data.DataLoader(
        val_subset,
        batch_size=int(config["batch_size"]),
        shuffle=True,
        num_workers=1)
    
    
    for epoch in range(5):
        
        running_loss = 0.0
        epoch_steps = 0
        
        for i, data in enumerate(trainloader,0):
            
            inputs,labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            epoch_steps += 1
            
            if i % 2000 == 1999:  # print every 2000 mini-batches
                print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1,
                                                running_loss / epoch_steps))
                running_loss = 0.0
                
                
            val_loss = 0.0
            val_steps = 0
            total = 0
            correct = 0
            
            for i, data in enumerate(valloader, 0):
                with torch.no_grad():
                    inputs, labels = data
                    inputs, labels = inputs.to(device), labels.to(device)

                    outputs = net(inputs)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

                    loss = criterion(outputs, labels)
                    val_loss += loss.cpu().numpy()
                    val_steps += 1
                
            with tune.checkpoint_dir(epoch) as checkpoint_dir:
                path = os.path.join(checkpoint_dir, "checkpoint")
                torch.save((net.state_dict(), optimizer.state_dict()), path)
            
            tune.report(loss=(val_loss / val_steps), accuracy=correct / total)
            
        
        print("Finished Training")    

In [20]:
def test_accuracy(net, device="cpu"):
    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2)

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total

In [21]:
config = {
    "l1": tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
    "l2": tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
    "lr": tune.loguniform(1e-4, 1e-1),
    "batch_size": tune.choice([2, 4, 8, 16])
}

In [22]:
gpus_per_trial = 2
scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=10,
        grace_period=1,
        reduction_factor=2)
reporter = CLIReporter(
        # parameter_columns=["l1", "l2", "lr", "batch_size"],
        metric_columns=["loss", "accuracy", "training_iteration"])
result = tune.run(
    partial(custom_training),
    resources_per_trial={"cpu": 2, "gpu": 1},
    config=config,
    num_samples=20,
    scheduler=scheduler,
    progress_reporter=reporter,
    )

== Status ==
Current time: 2022-08-30 16:12:09 (running for 00:00:00.25)
Memory usage on this node: 11.4/15.4 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 2.0/16 CPUs, 1.0/1 GPUs, 0.0/3.77 GiB heap, 0.0/1.88 GiB objects
Result logdir: C:\Users\danwe\ray_results\custom_training_2022-08-30_16-12-09
Number of trials: 17/20 (16 PENDING, 1 RUNNING)
+-----------------------------+----------+-----------------+--------------+------+------+-------------+
| Trial name                  | status   | loc             |   batch_size |   l1 |   l2 |          lr |
|-----------------------------+----------+-----------------+--------------+------+------+-------------|
| custom_training_bc76d_00000 | RUNNING  | 127.0.0.1:19976 |            2 |   32 |  256 | 0.0176463   |
| custom_training_bc76d_00001 | PENDING  |                 |           16 |    8 |    4 | 0.016166    |
| custom_training_bc76d_00002 | PEN

 87%|████████▋ | 8673280/9912422 [00:01<00:00, 10550192.31it/s]
9913344it [00:01, 7827228.90it/s]                              


(func pid=19976) Extracting data\MNIST\raw\train-images-idx3-ubyte.gz to data\MNIST\raw
(func pid=19976) 
(func pid=19976) Downloading http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
(func pid=19976) Downloading http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz to data\MNIST\raw\train-labels-idx1-ubyte.gz
(func pid=19976) Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz to data\MNIST\raw
(func pid=19976) 
(func pid=19976) Downloading http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz


29696it [00:00, ?it/s]                   


(func pid=19976) Downloading http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz to data\MNIST\raw\t10k-images-idx3-ubyte.gz


...

== Status ==
Current time: 2022-08-30 16:19:50 (running for 00:07:40.37)
Memory usage on this node: 11.1/15.4 GiB
Using AsyncHyperBand: num_stopped=19
Bracket: Iter 8.000: -2.2975736548900603 | Iter 4.000: -2.299649904489517 | Iter 2.000: -2.3056497256358464 | Iter 1.000: -2.3141428904533385
Resources requested: 2.0/16 CPUs, 1.0/1 GPUs, 0.0/3.77 GiB heap, 0.0/1.88 GiB objects
Result logdir: C:\Users\danwe\ray_results\custom_training_2022-08-30_16-12-09
Number of trials: 20/20 (1 RUNNING, 19 TERMINATED)
+-----------------------------+------------+-----------------+--------------+------+------+-------------+---------+------------+----------------------+
| Trial name                  | status     | loc             |   batch_size |   l1 |   l2 |          lr |    loss |   accuracy |   training_iteration |
|-----------------------------+------------+-----------------+--------------+------+------+-------------+---------+------------+----------------------|
| custom_training_bc76d_00019 | RUNN

Result for custom_training_bc76d_00019:
  accuracy: 0.1155
  date: 2022-08-30_16-20-00
  done: false
  experiment_id: fa4dc86d543c4a9aaf0a41dc0e13f258
  hostname: Gaming-Notebook
  iterations_since_restore: 3
  loss: 2.30022705078125
  node_ip: 127.0.0.1
  pid: 20216
  should_checkpoint: true
  time_since_restore: 16.669424057006836
  time_this_iter_s: 3.6253886222839355
  time_total_s: 16.669424057006836
  timestamp: 1661869200
  timesteps_since_restore: 0
  training_iteration: 3
  trial_id: bc76d_00019
  warmup_time: 0.005001068115234375
  
== Status ==
Current time: 2022-08-30 16:20:04 (running for 00:07:54.82)
Memory usage on this node: 11.1/15.4 GiB
Using AsyncHyperBand: num_stopped=19
Bracket: Iter 8.000: -2.2975736548900603 | Iter 4.000: -2.298770042737325 | Iter 2.000: -2.304155941406886 | Iter 1.000: -2.3137737923463186
Resources requested: 2.0/16 CPUs, 1.0/1 GPUs, 0.0/3.77 GiB heap, 0.0/1.88 GiB objects
Result logdir: C:\Users\danwe\ray_results\custom_training_2022-08-30_16-1

Result for custom_training_bc76d_00019:
  accuracy: 0.16025
  date: 2022-08-30_16-20-15
  done: false
  experiment_id: fa4dc86d543c4a9aaf0a41dc0e13f258
  hostname: Gaming-Notebook
  iterations_since_restore: 7
  loss: 2.294112848599752
  node_ip: 127.0.0.1
  pid: 20216
  should_checkpoint: true
  time_since_restore: 31.90082573890686
  time_this_iter_s: 3.5335631370544434
  time_total_s: 31.90082573890686
  timestamp: 1661869215
  timesteps_since_restore: 0
  training_iteration: 7
  trial_id: bc76d_00019
  warmup_time: 0.005001068115234375
  
== Status ==
Current time: 2022-08-30 16:20:19 (running for 00:08:10.25)
Memory usage on this node: 11.3/15.4 GiB
Using AsyncHyperBand: num_stopped=19
Bracket: Iter 8.000: -2.2934517860412598 | Iter 4.000: -2.298770042737325 | Iter 2.000: -2.304155941406886 | Iter 1.000: -2.3137737923463186
Resources requested: 2.0/16 CPUs, 1.0/1 GPUs, 0.0/3.77 GiB heap, 0.0/1.88 GiB objects
Result logdir: C:\Users\danwe\ray_results\custom_training_2022-08-30_16-1

2022-08-30 16:20:27,618	INFO tune.py:758 -- Total run time: 497.95 seconds (497.80 seconds for the tuning loop).


Result for custom_training_bc76d_00019:
  accuracy: 0.10933333333333334
  date: 2022-08-30_16-20-27
  done: true
  experiment_id: fa4dc86d543c4a9aaf0a41dc0e13f258
  hostname: Gaming-Notebook
  iterations_since_restore: 10
  loss: 2.2858972422281902
  node_ip: 127.0.0.1
  pid: 20216
  should_checkpoint: true
  time_since_restore: 43.393768310546875
  time_this_iter_s: 3.811290979385376
  time_total_s: 43.393768310546875
  timestamp: 1661869227
  timesteps_since_restore: 0
  training_iteration: 10
  trial_id: bc76d_00019
  warmup_time: 0.005001068115234375
  
== Status ==
Current time: 2022-08-30 16:20:27 (running for 00:08:17.80)
Memory usage on this node: 11.3/15.4 GiB
Using AsyncHyperBand: num_stopped=20
Bracket: Iter 8.000: -2.2934517860412598 | Iter 4.000: -2.298770042737325 | Iter 2.000: -2.304155941406886 | Iter 1.000: -2.3137737923463186
Resources requested: 2.0/16 CPUs, 1.0/1 GPUs, 0.0/3.77 GiB heap, 0.0/1.88 GiB objects
Result logdir: C:\Users\danwe\ray_results\custom_training_

In [ ]:
best_trial = result.get_best_trial("loss", "min", "last")

In [ ]:
best_trial

In [ ]:
print("Best trial config: {}".format(best_trial.config))
print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))
print("Best trial final validation accuracy: {}".format(
        best_trial.last_result["accuracy"]))

In [23]:
def load_data(data_dir="./data"):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    trainset = torchvision.datasets.CIFAR10(
        root=data_dir, train=True, download=True, transform=transform)

    testset = torchvision.datasets.CIFAR10(
        root=data_dir, train=False, download=True, transform=transform)

    return trainset, testset

In [24]:
class Net(nn.Module):
    def __init__(self, l1=120, l2=84):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, l1)
        self.fc2 = nn.Linear(l1, l2)
        self.fc3 = nn.Linear(l2, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [28]:
def train_cifar(config, checkpoint_dir=None, data_dir=None):
    net = Net(config["l1"], config["l2"])

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=config["lr"], momentum=0.9)

    if checkpoint_dir:
        model_state, optimizer_state = torch.load(
            os.path.join(checkpoint_dir, "checkpoint"))
        net.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)

    trainset, testset = load_data(data_dir)

    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs])

    trainloader = torch.utils.data.DataLoader(
        train_subset,
        batch_size=int(config["batch_size"]),
        shuffle=True,
        num_workers=2)
    valloader = torch.utils.data.DataLoader(
        val_subset,
        batch_size=int(config["batch_size"]),
        shuffle=True,
        num_workers=2)

    for epoch in range(10):  # loop over the dataset multiple times
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            epoch_steps += 1
            if i % 2000 == 1999:  # print every 2000 mini-batches
                print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1,
                                                running_loss / epoch_steps))
                running_loss = 0.0

        # Validation loss
        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        with tune.checkpoint_dir(epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((net.state_dict(), optimizer.state_dict()), path)

        tune.report(loss=(val_loss / val_steps), accuracy=correct / total)
    print("Finished Training")

In [29]:
def test_accuracy(net, device="cpu"):
    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2)

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total

In [30]:
config = {
    "l1": tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
    "l2": tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
    "lr": tune.loguniform(1e-4, 1e-1),
    "batch_size": tune.choice([2, 4, 8, 16])
}

In [32]:
def main(num_samples=10, max_num_epochs=10, gpus_per_trial=2):
    data_dir = os.path.abspath("./data")
    load_data(data_dir)
    config = {
        "l1": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
        "l2": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
        "lr": tune.loguniform(1e-4, 1e-1),
        "batch_size": tune.choice([2, 4, 8, 16])
    }
    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)
    reporter = CLIReporter(
        # parameter_columns=["l1", "l2", "lr", "batch_size"],
        metric_columns=["loss", "accuracy", "training_iteration"])
    result = tune.run(
        partial(train_cifar, data_dir=data_dir),
        resources_per_trial={"cpu": 2, "gpu": gpus_per_trial},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter)

    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))
    print("Best trial final validation accuracy: {}".format(
        best_trial.last_result["accuracy"]))

    best_trained_model = Net(best_trial.config["l1"], best_trial.config["l2"])
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if gpus_per_trial > 1:
            best_trained_model = nn.DataParallel(best_trained_model)
    best_trained_model.to(device)

    best_checkpoint_dir = best_trial.checkpoint.value
    model_state, optimizer_state = torch.load(os.path.join(
        best_checkpoint_dir, "checkpoint"))
    best_trained_model.load_state_dict(model_state)

    test_acc = test_accuracy(best_trained_model, device)
    print("Best trial test set accuracy: {}".format(test_acc))


if __name__ == "__main__":
    # You can change the number of GPUs per trial here:
    main(num_samples=10, max_num_epochs=10, gpus_per_trial=0)

NameError: name 'num_samples' is not defined